In [102]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
import base64

import os
import time
import gc
from collections import defaultdict

from officelib.xllib import *
from officelib import wordlib
from pywintypes import com_error

In [135]:
dl = os.path.expanduser("~\\downloads")
matrix_report_file = dl + "\\matrix-report.xlsx"
report_file = matrix_report_file.replace("report.xlsx", "dl-in.xlsx")

def del_old_reports():
    for file in (matrix_report_file, report_file):
        try:
            os.remove(file)
        except FileNotFoundError:
            pass  # fine with me

In [136]:
class wait_for_handle_if:
    """ Waits for a condition,
    but checks for another condition first
    and handles it if found.
    
    `handle_condition` must return a single object
    `func` must accept a single object returned from `handle_condition`. 
    
    """
    def __init__(self, wait_condition, handle_condition, func):
        self.wait = wait_condition
        self.handle = handle_condition
        self.func = func
        
    def __call__(self, driver):
        try:
            elem = self.handle(driver)
        except WebDriverException:
            pass
        else:
            self.func(elem)
        
        return self.wait(driver)
        

In [137]:
def dl_mtx_report(usr, pw, report_id):
    print("loading selenium driver")
    with webdriver.Chrome() as driver:
        _dl_mtx_report(driver, usr, pw, report_id)
        
def _dl_mtx_report(driver, usr, pw, report_id):
    
    print("opening page")
    driver.get(f"https://pbsbiotech.helixalm.cloud/ttweb/#default/10/previewReport/{report_id}")

    print("waiting for login page to dynamically load...")
    wait = WebDriverWait(driver, 15, 0.5)

    
    # Helix's login page uses dynamic loading, so wait until the login form is available.
    
    username = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "#login-username [name='login-username']")))
    password = driver.find_element_by_css_selector("#login-password [name='login-password']")
    log_in = driver.find_element_by_css_selector("label[title='Log in to Helix ALM']")

    # Using a previous EC wait method, the element would sometimes be selected by 
    # selenium before it could actually have text entered, resulting in a stall.
    
    # Method was changed and short wait added at the same time - unsure if wait is
    # actually necessary, but it is short enough to be not noticeable.
    
    time.sleep(0.2)
    
    print("logging in...")
    username.send_keys(usr)
    password.send_keys(pw)
    
    # grab handles before clicking to avoid race condition
    current_handles = driver.window_handles
    log_in.click()

    # clear the alert dialog if it appears for existing login
    # note: no clean way to get dialog button reference
    
    # DOM inspection shows this ID seems to be consistent for the
    # "Log out and start new session" button. I don't like this
    # but it seems to work. 
    
    print("waiting for report to load...")
    cond = wait_for_handle_if(EC.new_window_is_opened(current_handles), 
                              EC.visibility_of_element_located((By.ID,"sc6854")),
                             lambda elem: elem.click())
    
    wait.until(cond)
    login_handle, report_handle = driver.window_handles
    driver.switch_to.window(login_handle)
    driver.close()
    driver.switch_to.window(report_handle)
    
    print("downloading report")
    driver.execute_script("javascript:ExportMatrixToExcel('maintable')")

    # no clean way to wait for the file to download
    # easiest workaround is to try and rename it over and over
    # until it lets us (file exists & is no longer open)
    
    # this only works because it was deleted earlier in the script
    # and we always know the name it downloads the file as. 
    
    timeout = 10  # seconds
    end = time.time() + timeout
    report_file = matrix_report_file.replace("report.xlsx", "dl-in.xlsx")

    while True:
        try:
            os.rename(matrix_report_file, report_file)
        except (FileNotFoundError, PermissionError):
            if time.time() > end:
                raise
            time.sleep(0.1)
        else:
            break
    
    print("selenium operation complete")

In [138]:
def _stripify(c):
    if c is None: return ""
    t = c.strip().split(" ")
    return " ".join(t[2:])

def _unpack(row):
    return [c or "" for c in row]

def _parse_row(row):
    feature, fd, req, test = _unpack(row)
    return feature, test

def _superfit(c):
    c.ColumnWidth = 255
    c.AutoFit()

def _worksheet2(wb):
    if wb.Worksheets.Count < 2:
        return wb.Worksheets.Add()
    return wb.Worksheets(2)

def _borderify(rng):
    xl_edges = [
        xlc.xlEdgeLeft,
        xlc.xlEdgeTop,
        xlc.xlEdgeBottom,
        xlc.xlEdgeRight,
        xlc.xlInsideVertical,
        xlc.xlInsideHorizontal
    ]
    borders = rng.Borders
    for e in xl_edges:
        b = borders(e)
        b.LineStyle = xlc.xlContinuous
        b.ColorIndex = 0
        b.TintAndShade = 0
        b.Weight = xlc.xlThin
    
def _formatify(rng):
    rng.Font.Size = 8
    
def xfer(xl, wb, ws, cells, cr):
    # get data for copy to sheet2
    tl = cr("A2")
    br = tl.End(xlc.xlDown).GetOffset(0, 3)

    # (Feature, Functional Design, System Requirements, Test Cases)
    from_range = cr(tl, br)
    data = from_range.Value2
    
    # because the export is stupid, fix the data
    data = [list(map(_stripify, line)) for line in data]
    from_range.Value2 = data
    
    for col in from_range.Columns:
        _superfit(col)
    from_range.Rows.AutoFit()

    # paste data
    ws2 = _worksheet2(wb)
    cr2 = ws2.Cells.Range
    ws2.UsedRange.Clear()

    fmap = defaultdict(set)
    for row in data:
        feature, test = _parse_row(row)
        fmap[feature].add(test)
    
    paste_data = []
    for feature, tests in fmap.items():
        tests_list = "\n".join(sorted(filter(None, tests)))
        if not tests_list:
            pass
#             continue
        row = (feature, tests_list)
        paste_data.append(row)
    
    tl2 = cr2("A2")
    br2 = tl2.GetOffset(len(paste_data) - 1, len(paste_data[0]) - 1)
    
    data_range = cr2(tl2, br2)
    table_range = cr2("A1", br2)
    data_range.Value2 = paste_data
    
    cr2("A1:B1").Value2 = [("Feature", "Manufacturing & IQ/OQ Test Sections")]
    
    for col in (1,2):
        _superfit(ws2.Columns(col))
    
    _borderify(table_range)
    _formatify(data_range)
    
    data_range.Rows.AutoFit()
    data_range.VerticalAlignment = xlc.xlTop
    
    return table_range.Value2

In [139]:
def import_helix_mtrx(xl, filename):
    wb = xl.Workbooks.Open(filename)
    ws = wb.Worksheets(1)
    cells = ws.Cells
    cr = cells.Range

    with screen_lock(xl):
        return xfer(xl, wb, ws, cells, cr)

In [177]:
def close_wb_if_user_is_dumbass(xl, filename):
    for wb in xl.Workbooks:
        if wb.FullName == filename:
            print(">:( >:( >:(")
            wb.Close(False)
            return
    
def InchesToPoints(inch):
    return inch * 72
    
def _word_page_setup(d):
    setup = d.PageSetup
    setup.Orientation = wordlib.wdc.wdOrientPortrait
    setup.TopMargin = InchesToPoints(0.5)
    setup.BottomMargin = InchesToPoints(0.5)
    setup.LeftMargin = InchesToPoints(0.5)
    setup.RightMargin = InchesToPoints(0.5)
    setup.Gutter = InchesToPoints(0)
    setup.HeaderDistance = InchesToPoints(0.5)
    setup.FooterDistance = InchesToPoints(0.5)

def _pad(cs, l, r, t, b):
    cs.TopPadding = InchesToPoints(t)
    cs.BottomPadding = InchesToPoints(b)
    cs.LeftPadding = InchesToPoints(l)
    cs.RightPadding = InchesToPoints(r)
    
def _fix_table_style(t):
    tsc = t.Style.Table.Condition
    conditions = [
        "wdFirstRow",
        "wdLastRow",
        "wdOddRowBanding",
        "wdEvenRowBanding",
        "wdFirstColumn",
        "wdLastColumn",
        "wdOddColumnBanding",
        "wdEvenColumnBanding",
        "wdNECell",
        "wdNWCell",
        "wdSECell",
        "wdSWCell"
    ]
    for c in conditions:
        e = getattr(wordlib.wdc, c)
        cs = tsc(e)
        _pad(cs, 0.08, 0.08, 0.08, 0.08)
        
def _apply_borders(t):
    borders = [
        "wdBorderTop",
        "wdBorderLeft",
        "wdBorderBottom",
        "wdBorderRight",
        "wdBorderHorizontal",
        "wdBorderVertical"
    ]
    for b in borders:
        e = getattr(wordlib.wdc, b)
        border = t.Borders(e)
        border.LineStyle = wordlib.wdc.wdLineStyleSingle
        border.LineWidth = wordlib.wdc.wdLineWidth050pt
    
def _fix_tables_and_wait_for_word(d):
    timeout = time.time() + 5
    while time.time() < timeout:
        try:
            count = d.Tables.Count
        except com_error:
            time.sleep(0.1)
        else:
            if count > 0: 
                d.Tables(1).Delete()
    print("Word failed to respond :(")
    raise TimeoutError("Waiting for word")
        
def _paste_data(d, rng, data):
    t = d.Tables.Add(rng, len(data), len(data[0]))
    for row, (a,b) in zip(t.Rows, data):
        row.Cells(1).Range.Text = a or ""
        row.Cells(2).Range.Text = b or ""
    return t
        
def make_word_table(w, data):

    print("creating documnet...")
    d = w.Documents.Add()
    d.Styles("Normal").NoSpaceBetweenParagraphsOfSameStyle = True
    _word_page_setup(d)
    
    print("Pasting data...")
    t = _paste_data(d, d.Range(0), data)
    
    print("applying borders & table size...")
    _apply_borders(t)
    
    # column & table width must be in this order
    t.Columns(1).Width = InchesToPoints(2.5)
    t.PreferredWidth = InchesToPoints(7.5)
    t.AllowAutoFit = True
    
    
    print("applying row formatting...")
    rows = t.Rows
    rows.HeightRule = wordlib.wdc.wdRowHeightAtLeast
    rows.Height = InchesToPoints(0.01)

    header = rows(1).Range
    header.Font.Bold = True

    shading = header.Shading
    shading.Texture = wordlib.wdc.wdTextureNone
    shading.ForegroundPatternColor = wordlib.wdc.wdColorAutomatic
    shading.BackgroundPatternColor = -603923969

    print("Finishing up table styling...")
    pad = InchesToPoints(0.08)        
    for ob in (t, t.Style.Table):
        ob.TopPadding = pad
        ob.BottomPadding = pad
        ob.LeftPadding = pad
        ob.RightPadding = pad
        ob.AllowPageBreaks = True
    
def make_excel_matrix(xl, usr, pw, report_id=6):
    with screen_lock(xl):
        close_wb_if_user_is_dumbass(xl, report_file)
        del_old_reports()
        dl_mtx_report(usr, pw, report_id)
        return import_helix_mtrx(xl, report_file)
    

In [178]:
def main(usr, pw, report_id):
    xl = Excel()
    data = make_excel_matrix(xl, usr, pw, report_id)
    
    w = wordlib.Word()
    with screen_lock(w):
        make_word_table(w, data)

def main2():
    data = Excel().Selection.Value2
    w = wordlib.Word()
    make_word_table(w, data)

In [179]:
import gc

# this is obviously not secure but for scripting purposes
# at least it can't be read off the screen by a normal human

usr = "nstarkweather@pbsbiotech.com"
pw = base64.b64decode(b"c2FkYmVhcmJvbmViYXI=").decode('utf-8')
report_id = 8

main(usr, pw, report_id)
gc.collect()  # free COM

>:( >:( >:(
loading selenium driver
opening page
waiting for login page to dynamically load...
logging in...
waiting for report to load...
downloading report
selenium operation complete
creating documnet...
Pasting data...
applying borders & table size...
applying row formatting...
Finishing up table styling...


0

In [174]:
w = wordlib.Word()

# while w.Documents.Count:
#     w.ActiveDocument.Close(False)
w.ActiveDocument.Tables(1).Columns(1).Width = InchesToPoints(3)

del w
gc.collect()

20